In [1]:
import numpy as np
import pandas as pd


In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
df = pd.read_csv(url,header=None)

In [3]:
from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(df.iloc[:,1:])
Y = np.array(df.iloc[:,0])

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=1/3,stratify=Y)

In [6]:
from sklearn.cluster import KMeans
kmedias = KMeans(n_clusters=15).fit(X_train)
centros = kmedias.cluster_centers_

In [7]:
from sklearn.metrics.pairwise import euclidean_distances
distancias = euclidean_distances(centros)
sigma = sum(distancias)/(centros.shape[0]-1)

In [8]:
def mahalanobis(X,Y,S):
    return (X-Y) @ S @ (X-Y).T

S = np.cov(X,rowvar=False)
S_inv = np.linalg.inv(S)

In [60]:
X_mlp_train = np.zeros((X_train.shape[0], centros.shape[0]), dtype=float)
for i in range(X_train.shape[0]):
    for j in range(centros.shape[0]):
        X_mlp_train[i][j] = np.exp(-mahalanobis(X_train[i],centros[j],S_inv)/(2.0*(sigma[j]**2)))

In [61]:
S = np.cov(X,rowvar=False)
S_inv = np.linalg.inv(S)
(X_train[0]-centros[0]) @ S_inv @ (X_train[0]-centros[0]).T

19.789408688887022

In [62]:
X_mlp_test = np.zeros((X_test.shape[0], centros.shape[0]), dtype=float)
for i in range(X_test.shape[0]):
    for j in range(centros.shape[0]):
        X_mlp_test[i][j] = np.exp(-mahalanobis(X_test[i],centros[j],S_inv)/(2.0*(sigma[j]**2)))

In [63]:
from sklearn.preprocessing import LabelBinarizer
etiquetas = LabelBinarizer()
etiquetas.fit([1,2,3])
Ymlp = etiquetas.transform(Y)
Y_mlp_train = etiquetas.transform(Y_train)
Y_mlp_test = etiquetas.transform(Y_test)

In [14]:
from sklearn.linear_model import SGDRegressor
adaline = SGDRegressor(max_iter=5000)
Yp_test = np.zeros((Y_test.shape[0],3))
for neurona_salida in range(3):
    adaline.fit(X_mlp_train,Y_mlp_train[:,neurona_salida])
    Yp_test[:,neurona_salida] = adaline.predict(X_mlp_test)
aciertos = sum(np.argmax(Yp_test, axis=1) == np.argmax(Y_mlp_test,axis=1)) / Y_mlp_test.shape[0]
print("Tasa de aciertos =",aciertos*100, "%")

Tasa de aciertos = 58.333333333333336 %


Resultados

||3|4|5|6|7|8|9|10|11|12|13|14|15|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|Euclídea|1|1|1|1|1|1|1|1|1|1|1|1|1|
|Mahalanobis|0.48|0.46|0.46|0.46|0.48|0.5|0.51|0.51|0.5|0.45|0.48|0.5|0.58|

# Parte 2: MLP de salida

In [75]:
from sklearn.neural_network import MLPClassifier

net = MLPClassifier(hidden_layer_sizes=(10),activation="identity",solver="lbfgs",batch_size=1,max_iter=5000,momentum=0)

In [76]:
net.fit(X_mlp_train,Y_train)

MLPClassifier(activation='identity', alpha=0.0001, batch_size=1, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=10, learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [77]:
Y_predict = net.predict(X_mlp_test)

In [78]:
sum(Y_predict == Y_test)/Y_test.shape[0]

0.95

Tasa de acierto del 95% para tamaño de cluster 15 y distancia de Mahalanobis, usando LBFGS